In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import vasicek

In [3]:
# These values are from "Implementing Derivatives Models, Clewlow"
kappa = 0.15
volatility = 0.01
r0 = 0.05
theta = r0

In [4]:
model = vasicek.Vasicek(volatility, theta, kappa)

In [5]:
curve_initial = model.get_curve(0, r0)

In [6]:
curve_initial.get_df(1)

0.9512436133209152

In [7]:
curve_initial.get_df(5)

0.7797609496298229

# Calculation setting

In [160]:
T = 10
timestep = 1/12
numgrid = (int)(T / timestep)
numpath = 10000

In [161]:
numgrid

120

# Generate Path

In [162]:
paths = np.ones((numpath, 1)) * r0

In [163]:
for i in range(numgrid):
    evolved = model.proceed(paths[:, i], timestep, np.random.normal(size = numpath))
    paths = np.append(paths, evolved.reshape(numpath, -1), axis = 1)
    

In [164]:
df_T = curve_initial.get_df(T)

In [165]:
t = 5
nt = (int)(numgrid * t / T)
path_on_t = paths[:, nt]

In [166]:
dfs_t = []
for r in path_on_t:
    curve = model.get_curve(t, r)
    dfs_t.append(curve.get_df(T))
    

In [167]:
# simulated DF(t, t, T)
dfttT = np.mean(dfs_t)
print('DF(t,t,T) = ', dfttT)
print('ZeroRate', -np.log(dfttT) / (T-t))

DF(t,t,T) =  0.7811799485778099
ZeroRate 0.04938994955442982


In [168]:
# Expectation DF(0, t, T)
df0tT = curve_initial.get_df(T) / curve_initial.get_df(t)
print('DF(0,t,T) = ', df0tT)
print('ZeroRate', -np.log(df0tT) / (T-t))

DF(0,t,T) =  0.7827126408493633
ZeroRate 0.048997929597910786


↑ずれている
時間のグリッドが荒いのが原因と考えられる

グリッドの切り方に応じてドリフトを調整する必要がある

## Pricing Discount Bond Option with Monte Carlo

In [171]:
option_expiry = 1
bond_maturity = 5
strike = 0.67

In [175]:
i_expiry = (int)(numgrid * option_expiry / T)
r_on_expiry = paths[:, i_expiry]

In [176]:
bond_prices = []
for r in r_on_expiry:
    curve = model.get_curve(option_expiry, r)
    bond_prices.append(curve.get_df(bond_maturity))

In [183]:
payoffs = np.maximum(0, np.array(bond_prices) - strike)
price = curve_initial.get_df(option_expiry) * payoffs.mean()

In [184]:
print('option price :', price)

option price : 0.14205194386903197


## FRA

In [185]:
start = 2
end = 2.5

In [189]:
def fra(curve):
    return (curve.get_df(start) / curve.get_df(end) - 1) / (end - start)

In [190]:
fra_initial = fra(curve_initial)

In [192]:
i_start = (int)(numgrid * start / T)
r_on_start = paths[:, i_start]

In [193]:
fras = []
for r in r_on_start:
    curve = model.get_curve(start, r)
    fras.append(fra(curve))

In [196]:
fra_mc = np.mean(fras)

In [197]:
print('analytic :', fra_initial)
print('monte carlo :', fra_mc)

analytic : 0.05044296909829926
monte carlo : 0.05071974577182463
